In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append("../")
from utils.helper_function import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

import optuna
import pickle
import datetime

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas")
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")

/tmp/ipykernel_13997/2887699915.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
xls = pd.ExcelFile('../data/data_sixpk.xlsx')
df = pd.read_excel(xls, 'AcuteInhal')
df.head()

,CASRN,SMILES,Activity
0,100-00-5,[O-][N+](=O)C1=CC=C(Cl)C=C1,1
1,100-20-9,ClC(=O)C1=CC=C(C=C1)C(Cl)=O,1
2,100-37-8,CCN(CC)CCO,1
3,100-43-6,C=CC1=CC=NC=C1,1
4,100-44-7,ClCC1=CC=CC=C1,1


In [3]:
X,y = df["SMILES"], df['Activity']

In [4]:
idx = valid_indices(X)
X = X[idx]
y = y[idx]

In [5]:
maccs = maccs_fp(X)


In [6]:
maccs = np.array(maccs)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(maccs, y, test_size= 0.2, random_state= 42)

In [7]:
def svm_modl(trial, xtrain, ytrain):
    n = trial.suggest_int('degree', 2, 5)
    model_svm = SVC(degree= n)

    scores = cross_validate(model_svm, xtrain, ytrain, cv=5, scoring='roc_auc')
    mean_roc = scores['test_score'].mean()

    return 1/(mean_roc + 1e-6)

In [8]:
study = optuna.create_study(study_name='SVM_study_acute_inhal', storage= None)  # Create a new study.
study.optimize(lambda trial: svm_modl(trial, X_train, y_train), n_trials=20)  # Invoke optimization of the objective function.

[I 2024-01-29 17:00:01,828] A new study created in memory with name: SVM_study_acute_inhal
[I 2024-01-29 17:00:02,060] Trial 0 finished with value: 1.2870656259222573 and parameters: {'degree': 2}. Best is trial 0 with value: 1.2870656259222573.
[I 2024-01-29 17:00:02,272] Trial 1 finished with value: 1.2870656259222573 and parameters: {'degree': 4}. Best is trial 0 with value: 1.2870656259222573.
[I 2024-01-29 17:00:02,480] Trial 2 finished with value: 1.2870656259222573 and parameters: {'degree': 2}. Best is trial 0 with value: 1.2870656259222573.
[I 2024-01-29 17:00:02,696] Trial 3 finished with value: 1.2870656259222573 and parameters: {'degree': 5}. Best is trial 0 with value: 1.2870656259222573.
[I 2024-01-29 17:00:02,913] Trial 4 finished with value: 1.2870656259222573 and parameters: {'degree': 3}. Best is trial 0 with value: 1.2870656259222573.
[I 2024-01-29 17:00:03,124] Trial 5 finished with value: 1.2870656259222573 and parameters: {'degree': 3}. Best is trial 0 with value:

In [9]:
best_model = SVC(degree= study.best_params['degree'])
best_model.fit(X_train, y_train)

SVC(degree=2)

In [10]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_true= y_test, y_pred= y_pred)
print(accuracy)

0.7572254335260116


In [11]:
tn, fp, fn, tp = confusion_matrix(y_pred, y_test).ravel()

ACC = (tp + tn)/(tp + tn + fn + fp)
SEN = tp/(tp + fn)
SPE = tn/(tn + fp)

print(f'Accuracy = {ACC}')
print(f'Sensitivity = {SEN}')
print(f'Specificity = {SPE}')


Accuracy = 0.7572254335260116
Sensitivity = 0.7692307692307693
Specificity = 0.7473684210526316


In [12]:
model_name = f"SVM_MACCS_model_Acute_Inhalation{datetime.datetime.now().isoformat()}"
model_path = "../models/"
file = os.path.join(model_path, model_name)
pickle.dump(best_model, open(file, 'wb'))